In [ ]:
!apt-get install -y xvfb
!pip install pyvirtualdisplay imageio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 18 not upgraded.
Need to get 7,815 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5build4 [172 kB]
Get:5 http://archiv

In [ ]:
# Import required modules
from pyvirtualdisplay import Display
import pygame
import random
import math
import numpy as np
import imageio
from IPython.display import Image, display as ipy_display

# ----- Simulation Constants -----
WIDTH, HEIGHT = 800, 600           # Size of the simulation window
NUM_INITIAL_ORGANISMS = 20         # Starting number of organisms
NUM_INITIAL_FOOD = 100             # Starting food pieces on the field
FOOD_ENERGY = 20                   # Energy gained when eating food
ORGANISM_INITIAL_ENERGY = 50       # Starting energy for organisms
ENERGY_DECAY = 0.05                # Energy lost per frame (movement cost)
BASE_METABOLISM = 0.01             # Constant energy cost per frame
REPRODUCTION_ENERGY_THRESHOLD = 100  # Energy level required to reproduce
MUTATION_RATE = 0.3                # Chance for a genetic mutation during reproduction
MUTATION_AMOUNT = 0.2              # How much the trait can change with mutation

# ----- Organism Class -----
class Organism:
    def __init__(self, x, y, speed, energy=ORGANISM_INITIAL_ENERGY):
        """
        Each organism has a position (x, y), a movement direction, a speed trait,
        an energy level, and a vision radius.
        """
        self.x = x
        self.y = y
        self.speed = speed  # This is the genetic trait that may mutate.
        self.energy = energy
        self.direction = random.uniform(0, 2 * math.pi)
        self.vision = 50 + speed * 10  # Vision range
        self.size = max(3, int(4 + speed * 2))  # Visual size of the organism

    def update(self, food_list):
        """Update organism behavior for one simulation frame."""
        closest_food = None
        min_dist = float('inf')
        # Look for the closest food within vision range.
        for food in food_list:
            dx = food.x - self.x
            dy = food.y - self.y
            dist = math.hypot(dx, dy)
            if dist < self.vision and dist < min_dist:
                min_dist = dist
                closest_food = food

        if closest_food:
            # Adjust direction to head toward the food.
            self.direction = math.atan2(closest_food.y - self.y, closest_food.x - self.x)
        else:
            # Wander randomly if no food is nearby.
            self.direction += random.uniform(-0.2, 0.2)

        # Move according to current direction and speed.
        self.x += math.cos(self.direction) * self.speed
        self.y += math.sin(self.direction) * self.speed

        # Wrap around the edges.
        if self.x < 0:
            self.x += WIDTH
        if self.x > WIDTH:
            self.x -= WIDTH
        if self.y < 0:
            self.y += HEIGHT
        if self.y > HEIGHT:
            self.y -= HEIGHT

        # Consume energy based on movement and a constant metabolic cost.
        self.energy -= ENERGY_DECAY * self.speed + BASE_METABOLISM

        # Check for collision with food and eat if close enough.
        if closest_food and min_dist < self.size + 2:
            self.eat(closest_food, food_list)

    def eat(self, food, food_list):
        """Increase energy by consuming food, and remove the food item."""
        self.energy += FOOD_ENERGY
        if food in food_list:
            food_list.remove(food)

    def can_reproduce(self):
        """Return True if the organism has enough energy to reproduce."""
        return self.energy >= REPRODUCTION_ENERGY_THRESHOLD

    def reproduce(self):
        """
        Create an offspring organism.
        The child inherits the parent's speed with a chance of mutation.
        Energy is split between parent and child.
        """
        if random.random() < MUTATION_RATE:
            child_speed = self.speed + random.uniform(-MUTATION_AMOUNT, MUTATION_AMOUNT)
            child_speed = max(0.5, child_speed)
        else:
            child_speed = self.speed

        self.energy /= 2  # Share energy with the child.
        return Organism(self.x, self.y, child_speed, energy=self.energy)

    def is_dead(self):
        """Return True if the organism's energy has dropped to zero or below."""
        return self.energy <= 0

    def draw(self, surface):
        """
        Draw the organism on the simulation window.
        Color is determined by its speed trait (faster = more red, slower = more blue).
        """
        speed_normalized = max(0, min(1, (self.speed - 0.5) / 4.5))
        red = int(255 * speed_normalized)
        blue = 255 - red
        color = (red, 100, blue)
        pygame.draw.circle(surface, color, (int(self.x), int(self.y)), self.size)

# ----- Food Class -----
class Food:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def draw(self, surface):
        """Draw the food as a small green circle."""
        pygame.draw.circle(surface, (0, 255, 0), (int(self.x), int(self.y)), 3)

# ----- Set Up Virtual Display -----
display = Display(visible=0, size=(WIDTH, HEIGHT))
display.start()

# ----- Initialize Pygame -----
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Evolving Ecosystem Simulator - Headless Mode")
clock = pygame.time.Clock()

# ----- Create Initial Populations -----
organisms = [Organism(random.uniform(0, WIDTH), random.uniform(0, HEIGHT), random.uniform(1, 4))
             for _ in range(NUM_INITIAL_ORGANISMS)]
food_list = [Food(random.uniform(0, WIDTH), random.uniform(0, HEIGHT))
             for _ in range(NUM_INITIAL_FOOD)]

# ----- Simulation Loop -----
frames = []         # List to store captured frames
max_frames = 600    # Maximum number of frames to run the simulation

try:
    for frame_count in range(max_frames):
        # Process events (though no interactive window will appear in Colab)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                raise KeyboardInterrupt

        # Update each organism.
        for organism in organisms[:]:
            organism.update(food_list)
            if organism.can_reproduce():
                organisms.append(organism.reproduce())
            if organism.is_dead():
                organisms.remove(organism)

        # Occasionally add new food.
        if random.random() < 0.1:
            food_list.append(Food(random.uniform(0, WIDTH), random.uniform(0, HEIGHT)))

        # Drawing section.
        screen.fill((0, 0, 0))  # Black background.
        for food in food_list:
            food.draw(screen)
        for organism in organisms:
            organism.draw(screen)

        # Display simulation info.
        font = pygame.font.SysFont(None, 24)
        info_text = font.render(f"Organisms: {len(organisms)}  Food: {len(food_list)}", True, (255, 255, 255))
        screen.blit(info_text, (10, 10))

        pygame.display.flip()
        clock.tick(60)  # 60 frames per second

        # Capture the frame.
        frame_data = pygame.surfarray.array3d(screen)
        frame_data = np.transpose(frame_data, (1, 0, 2))
        frames.append(frame_data)
except KeyboardInterrupt:
    print("Simulation interrupted by user.")
finally:
    pygame.quit()

# ----- Save Captured Frames as an Animated GIF -----
gif_filename = 'simulation.gif'
imageio.mimsave(gif_filename, frames, fps=60)

# ----- Display the GIF in the Notebook -----
ipy_display(Image(gif_filename))